In [3]:
!pip install pytorch-lightning segmentation_models_pytorch tacoreader rasterio albumentations --upgrade

  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.8/58.8 kB 5.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 121.3/121.3 kB 12.1 MB/s eta 0:00:00
  Created wheel for efficientnet-pytorch: filename=efficientnet_pytorch-0.7.1-py3-none-any.whl size=16424 sha256=8dfb1e0126e83f8301f6d196d11ce896ce0ae4263839296bd9dcb80e2f938452
  Stored in directory: /root/.cache/pip/wheels/8b/6f/9b/231a832f811ab6ebb1b32455b177ffc6b8b1cd8de19de70c09
  Created wheel for pretrainedmodels: filename=pretrainedmodels-0.7.4-py3-none-any.whl size=60944 sha256=435fd42b8176f7b5cafe12d6dc3780147631fb5575e7e8fb15d3ae901f890337
  Stored in directory: /root/.cache/pip/wheels/5f/5b/96/fd94bc35962d7c6b699e8814db545155ac91d2b95785e1b035
Successfully built efficientnet-pytorch pretrainedmodels


In [1]:
import pytorch_lightning as pl
from pytorch_lightning.loggers import TensorBoardLogger
from datamodule import CoreDataModule
from model import UNet_CD_Sentinel_2
import config
import metrics
import tacoreader
import torch

In [2]:
# Primeiro Treinamento com as imagens 512x512
tb_logger = TensorBoardLogger(config.DIR_LOG, name=config.NAME_EFFICIENTNETB1)

dataset_l1c = tacoreader.load(["/content/drive/MyDrive/taco_CloudSen12/cloudsen12-l1c.0000.part.taco",
                              "/content/drive/MyDrive/taco_CloudSen12/cloudsen12-l1c.0001.part.taco",
                              "/content/drive/MyDrive/taco_CloudSen12/cloudsen12-l1c.0002.part.taco",
                              "/content/drive/MyDrive/taco_CloudSen12/cloudsen12-l1c.0003.part.taco",
                              "/content/drive/MyDrive/taco_CloudSen12/cloudsen12-l1c.0004.part.taco",
                              ])
df_512 = dataset_l1c[(dataset_l1c["label_type"] == "high") & (dataset_l1c["real_proj_shape"] == 509)]

# Define the datamodule
datamodule = CoreDataModule(
    dataframe=df_512,
    batch_size=config.BATCH_SIZE
)

# Define the model
model = UNet_CD_Sentinel_2(
    encoder_name=config.ENCODER_NAME_EFFICIENTNETB1,
    classes=config.CLASSES,
    in_channels=config.IN_CHANNELS,
    learning_rate=config.LEARNING_RATE,
)

checkpoint_callback = pl.callbacks.ModelCheckpoint(
    dirpath=config.DIR_ROOT_EFFICIENTNETB1,
    filename="{epoch}-{train_loss:.2f}-{val_loss:.2f}-trainHigh512",
    monitor="val_loss",
    mode="min",
    save_top_k=1,
)

earlystopping_callback = pl.callbacks.EarlyStopping(
    monitor="val_loss", patience=12, mode="min"
)

callbacks = [checkpoint_callback, earlystopping_callback]

# Define the trainer
trainer = pl.Trainer(
    max_epochs=config.EPOCHS,
    log_every_n_steps=1,
    callbacks=callbacks,
    accelerator=config.ACCELERATOR,
    precision="16-mixed",
    logger=tb_logger,
    default_root_dir=config.DIR_ROOT_EFFICIENTNETB1,
)


INFO:pytorch_lightning.utilities.rank_zero:Using 16bit Automatic Mixed Precision (AMP)
INFO:pytorch_lightning.utilities.rank_zero:GPU available: True (cuda), used: True
INFO:pytorch_lightning.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO:pytorch_lightning.utilities.rank_zero:HPU available: False, using: 0 HPUs


In [3]:
torch.set_float32_matmul_precision('medium')
# Start the training
trainer.fit(model=model, datamodule=datamodule)

/usr/local/lib/python3.11/dist-packages/pytorch_lightning/callbacks/model_checkpoint.py:654: Checkpoint directory /content/drive/MyDrive/Unet_all_bands_l1c/lightning_logs/Unet_efficientnet-b1 exists and is not empty.
INFO:pytorch_lightning.accelerators.cuda:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
INFO:pytorch_lightning.callbacks.model_summary:
  | Name  | Type             | Params | Mode 
---------------------------------------------------
0 | model | Unet             | 8.8 M  | train
1 | loss  | CrossEntropyLoss | 0      | train
---------------------------------------------------
8.8 M     Trainable params
0         Non-trainable params
8.8 M     Total params
35.042    Total estimated model params size (MB)
425       Modules in train mode
0         Modules in eval mode


Sanity Checking: |          | 0/? [00:00<?, ?it/s]

Training: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

...

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

In [4]:
# Carregar o melhor modelo diretamente
model = UNet_CD_Sentinel_2.load_from_checkpoint(
    checkpoint_callback.best_model_path,
    encoder_name=config.ENCODER_NAME_EFFICIENTNETB1,
    classes=config.CLASSES,
    in_channels=config.IN_CHANNELS,
    learning_rate=config.LEARNING_RATE,
)

# run val dataset
val_metrics = trainer.validate(model, datamodule=datamodule, verbose=True)
print(val_metrics)

# run test dataset
test_metrics = trainer.test(model, datamodule=datamodule, verbose=True)
print(test_metrics)

acuracia, iou, f1_score, recall, precision, sensitivity = metrics.calculate_metrics(datamodule.test_dataloader(), model.model)

# Salva o modelo treinado
smp_model = model.model
smp_model.save_pretrained(config.DIR_ROOT_EFFICIENTNETB1 + "/" + config.NAME_EFFICIENTNETB1)

INFO:pytorch_lightning.accelerators.cuda:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


Validation: |          | 0/? [00:00<?, ?it/s]

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃      Validate metric      ┃       DataLoader 0        ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│         val_loss          │    0.2429846078157425     │
│      valid_acuracia       │    0.9530059695243835     │
│     valid_dataset_iou     │    0.8281733989715576     │
│      valid_f1_score       │    0.9060118198394775     │
└───────────────────────────┴───────────────────────────┘

INFO:pytorch_lightning.accelerators.cuda:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


[{'val_loss': 0.2429846078157425, 'valid_acuracia': 0.9530059695243835, 'valid_dataset_iou': 0.8281733989715576, 'valid_f1_score': 0.9060118198394775}]


Testing: |          | 0/? [00:00<?, ?it/s]

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃        Test metric        ┃       DataLoader 0        ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│       test_acuracia       │    0.9527848362922668     │
│     test_dataset_iou      │     0.827434778213501     │
│       test_f1_score       │    0.9055696725845337     │
│         test_loss         │    0.24968993663787842    │
└───────────────────────────┴───────────────────────────┘

[{'test_loss': 0.24968993663787842, 'test_acuracia': 0.9527848362922668, 'test_dataset_iou': 0.827434778213501, 'test_f1_score': 0.9055696725845337}]


Calculando métricas: 100%|██████████| 31/31 [08:20<00:00, 16.16s/it]


Redução: micro
Acurácia no conjunto de teste: 0.9528
IoU no conjunto de teste: 0.8274
F1 no conjunto de teste: 0.9056
Recall no conjunto de teste: 0.9056
Precisião no conjunto de teste: 0.9056
Sensibilidade no conjunto de teste: 0.9056


Redução: macro
Acurácia no conjunto de teste: 0.9528
IoU no conjunto de teste: 0.7400
F1 no conjunto de teste: 0.8425
Recall no conjunto de teste: 0.8510
Precisião no conjunto de teste: 0.8346
Sensibilidade no conjunto de teste: 0.8510


Redução: micro-imagewise
Acurácia no conjunto de teste: 0.9528
IoU no conjunto de teste: 0.8436
F1 no conjunto de teste: 0.9056
Recall no conjunto de teste: 0.9056
Precisião no conjunto de teste: 0.9056
Sensibilidade no conjunto de teste: 0.9056


Redução: macro-imagewise
Acurácia no conjunto de teste: 0.9528
IoU no conjunto de teste: 0.7583
F1 no conjunto de teste: 0.8139
Recall no conjunto de teste: 0.8598
Precisião no conjunto de teste: 0.8694
Sensibilidade no conjunto de teste: 0.8598


